In [ ]:
import sys
import nivapy
import qgrid
import pandas as pd

# Quick hack to import from relative folder
sys.path.insert(0, '../ndbview')
from ndbview import ndb_queries

# Testing queries for NDBView

`ndb_queries.py` contains functions used by the Flask end points in `ndbview.py` to query the NIVADATABASE. The code is structured so that, as far as possible, the application-specific code is all in `ndbview.py`, and `ndb_queries.py` provides a basic database API that can also be used from Jupyter. This is useful for testing, and will eventually be useful for [NivaPy](https://github.com/NIVANorge/nivapy) too.

## 1. Connect to NIVADATABASE

In [ ]:
# Connect to db
ora_eng = nivapy.da.connect()

## 2. User defined parameters

In [ ]:
# Period of interest
st_dt = '1900-01-01'
end_dt = '2018-12-31'

# Include LOD flags ('<' or '>') in output?
lod_flags = False

Now complete **either** step 3.1 to query by projects, or skip ahead to step 3.2 to query the stations directly. **Choose either step 3.1 *or* step 3.2, not both**.

## 3.1. Query by project

### 3.1.1. Select projects of interest

In [ ]:
# List projects
prj_df = ndb_queries.get_all_projects(ora_eng)

# Convert to qgrid
prj_grid = qgrid.QgridWidget(df=prj_df, show_toolbar=False)
prj_grid

In [ ]:
# Show selected projects
sel_prj = prj_grid.get_selected_df()
sel_prj

### 3.1.2. Get stations for selected project(s)

In [ ]:
# Get stations for selected project
sel_prj_ids = sel_prj[['project_id']].copy()
sel_prj_ids.columns = ['id',]
stn_df = ndb_queries.get_project_stations(sel_prj_ids, ora_eng)

# Convert to qgrid
stn_grid = qgrid.QgridWidget(df=stn_df, show_toolbar=False)

print 'Number of unique station IDs:', len(stn_df)
stn_grid

### 3.1.3. Refine station selection

If desired, make a final sub-selection of the stations of interest (or select none to include all stations for these projects).

In [ ]:
# Get selected stations
if len(stn_grid.get_selected_df()) == 0:
    sel_stn = stn_df
else:
    sel_stn = stn_grid.get_selected_df()

print 'Number of stations selected:', len(sel_stn)
sel_stn

## 3.2. Select stations directly

This section is only relevant if you skipped over section 3.1.

In [ ]:
# List stations
stn_df = ndb_queries.get_all_stations(ora_eng)

# Convert to qgrid
stn_grid = qgrid.QgridWidget(df=stn_df, show_toolbar=False)
stn_grid

In [ ]:
# Show selected stations
sel_stn = stn_grid.get_selected_df()
sel_stn

## 4. Get parameters for selected stations and time period

In [ ]:
# Get parameters
sel_stn_ids = sel_stn[['station_id']].copy()
sel_stn_ids.columns = ['id',]
par_df = ndb_queries.get_station_parameters2(sel_stn_ids, st_dt, end_dt, ora_eng)

# Convert to qgrid
par_grid = qgrid.QgridWidget(df=par_df, show_toolbar=False)

print '%s parameters available.' % len(par_df)
par_grid

In [ ]:
# Show selected parameters
sel_par = par_grid.get_selected_df()
sel_par

## 5. Get water chemistry

In [ ]:
# Get water chemsitry
sel_stn_ids.columns = ['stns',]
sel_par_ids = sel_par[['parameter_id']].copy()
sel_par_ids.columns = ['pars',]
wc_df, dup_df = ndb_queries.get_chemistry_values2(sel_stn_ids, sel_par_ids, 
                                                  st_dt, end_dt, 
                                                  lod_flags, ora_eng)

print 'Number of duplicated records:', len(dup_df)
print 'Number of unique samples:', len(wc_df)

In [ ]:
# Convert duplicates to qgrid
dup_grid = qgrid.QgridWidget(df=dup_df, show_toolbar=False)
dup_grid

In [ ]:
# Convert chem to qgrid
wc_grid = qgrid.QgridWidget(df=wc_df, show_toolbar=False)
wc_grid

In [ ]:
import numpy as np

In [ ]:
def interpolate_water_chemistry(wc_df, freq='D', agg_stat='mean', 
                                interp_type='linear', interp_limit=None):
    """ Interpolate water chemistry data to a regular frequency.
    
        Performs interpolation for each unique station-sample depth 
        combination in wc_df. Each sub-dataset is first resampled to the
        specified (regular) frequency by applying the function supplied as
        'agg_stat'. The resampled dataframe is then interpolated using
        'interp_type'.
        
        For example, if wc_df contains data for a single station, but from
        two different depths, the series for each depth will be treated
        separately. Specifying freq='M', agg_stat='sum', interp_type='cubic'
        and interp_limit=6 will do the following:
        
            1. Resample the data from each depth to a regular monthly series 
               by summing all values within each month
               
            2. Interpolate values for months with no data using cubic splines.
               Data gaps of greater than 6 months will be left unfilled.
               
        Args:
            wc_df:        Dataframe. Water chemistry returned by 
                          get_chemistry_values2()
            freq:         Str. {'D':daily, 
                                'M':monthly,
                                'Q-NOV':seasonal,
                                'A':annual}
            agg_stat:     Str. One of ('mean', 'median', 'sum')
            interp_type:  Str. One of ('nearest', 'linear', 'cubic', 'quadratic')
            interp_limit: Int. The maximum data gap (number of steps at the 
                          resampled frequency) to interpolate over
        
        Returns:
            Dataframe of interpolated values for each station-depth combination        
    """
    # Check user input
    assert freq in ('D', 'M', 'Q-NOV', 'A'), (
           "'freq' argument is not valid.")
    assert agg_stat in ('mean', 'median', 'sum'), (
           "'agg_stat' argument is not valid.")
    assert interp_type in ('nearest', 'linear', 'cubic', 'quadratic'), (
           "'interp_type' argument not valid.")
    assert (isinstance(interp_limit, int) or interp_limit == None), (
           "'interp_limit' must be an integer number of steps.")

    # Container for data
    df_list = []

    # Get depth groups
    depth_grps = wc_df.groupby(['depth1', 'depth2']).groups.keys()

    # treat each station and depth combo separately
    for dep1, dep2 in depth_grps:
        for stn_id in wc_df['station_id'].unique():
            # Get the stn data
            df = wc_df.query('(station_id == @stn_id) and '
                             '(depth1 == @dep1) and '
                             '(depth2 == @dep2)')

            # If not emoty df
            if len(df) > 0:
                # Remove cols we don't want to aggregate
                del df['station_id'], df['depth1'], df['depth2']
                
                # Set index
                df.set_index('sample_date', inplace=True)

                # Resample
                if agg_stat == 'mean':
                    df = df.resample(freq).mean().to_period()
                elif agg_stat == 'median':
                    df = df.resample(freq).median().to_period()
                elif agg_stat == 'sum':
                    df = df.resample(freq).sum().to_period()
                else:
                    raise ValueError("'agg_stat' must be one of "
                                     "('mean', 'median', or 'sum')")

                # Interpolate
                if len(df) > 1:
                    df.interpolate(method=interp_type, 
                                   limit=interp_limit, 
                                   inplace=True)

                # Add index cols again
                df['station_id'] = stn_id
                df['depth1'] = dep1
                df['depth2'] = dep2
                
                # Tidy
                df.reset_index(inplace=True)
                cols1 = ['station_id', 'sample_date', 'depth1', 'depth2']
                cols2 = [col for col in df.columns if col not in cols1]
                df = df[cols1 + cols2]

                # Add to output
                df_list.append(df)

    # Combine results
    df = pd.concat(df_list, axis=0)
    df.reset_index(inplace=True)

    return df

In [ ]:
# Interpolate
df = interpolate_water_chemistry(wc_df, freq='A', agg_stat='median', 
                                 interp_type='cubic', interp_limit=None)

# Convert duplicates to qgrid
interp_grid = qgrid.QgridWidget(df=df, show_toolbar=False)
interp_grid